<a href="https://colab.research.google.com/github/mdallas1/shared_code/blob/main/L5_2_lu_with_pivoting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! apt install octave

# Trouble with the LU method

In the last lecture, we saw that we can efficiently solve an $n\times n$ nonsingular linear system $A\mathbf{x}=\mathbf{b}$ by first computing the LU factorization $A=LU$, where $L$ is lower triangular and $U$ is upper triangular, and then solving $L\mathbf{y} = \mathbf{b}$ with forward subsitution, and $U\mathbf{x} = \mathbf{y}$ with backward subsitution. This is great, but it assumes that we can actually compute the LU factorization of $A$ with the algorithms introduced in the previous lecture. Even if $A$ is nonsingular, our algorithm may break down, meaning we cannot compute $A=LU$. Take
$$A = \begin{bmatrix} 1 &1 &4 \\ 2 &2 &1\\ 0 &1 &0\end{bmatrix}$$

Let's apply our algorithm from last time to try and compute $L$ and $U$ for this matrix $A$.

$$L_1A = \begin{bmatrix} 1 &0 &0 \\ -2 &1 &0\\ 0 &0 &1\end{bmatrix} \begin{bmatrix} 1 &1 &4 \\ 2 &2 &1\\ 0 &1 &0\end{bmatrix} = \begin{bmatrix} 1 &1 &4 \\ 0 &0 &-7\\ 0 &1 &0\end{bmatrix}$$

We're almost done, we just need to turn the 1 in the third row, second column into a 0. Our algorithm from last time calls for us to apply Gauss elimination using the diagonal as a pivot, but then we need to divide by the entry in the second row and second column...which is zero. This means our algorithm breaks down, and we are unable to compute $L$ and $U$, so we can't solve this sytem with the $LU$ method, even though $A$ is nonsingular!




# When does $A$ have an $LU$ factorization?

This begs the question: for which matrices will our LU algorithm work? In other words, what matrices have LU factorizations? This is answered by the following proposition. Note that the $i$th *principle submatrix* of a matrix $A$ is the submatrix formed by taking the first $i$ rows and $i$ columns, and is denoted by $A_i$. This is Proposition 5.1 in our text.

> **Proposition 5.1** For a given $n\times n$ real matrix $A$,  an LU factorization exists and is unique if and only if the principle submatrices $A_i$ for $i=1,...,n-1$ are nonsingular.

If you look at the example above in which we failed to find an LU factorization, you'll note that $$A_2 =\begin{bmatrix} 1 &1 \\ 2 &2\end{bmatrix}$$ is singular. So in hindsight it's no suprise that our algorithm broke down.

The hypothses of Proposition 5.1 can be tedious to check. There some special cases where it can be shown to hold under simpler conditions. The following classes of matrices are all guaranteed to have a unique LU factorization.

1. *Stricly diagonally dominant matrices*. $A$ is a strictly diagonally dominant matrix *by row* if
$$ |a_{ii}| > \sum_{j=1, j\neq i}^n |a_{ij}|$$, and
stricly diagonally dominant *by column* if
$$ |a_{jj}| > \sum_{i=1, i\neq j}^n |a_{ij}|$$.

An example of a $4\times 4$ matrix that is strictly diagonally dominant by row is

$$A = \begin{bmatrix} 10 &1 &-2 &3 \\ -2 &11 &-4 &2 \\ 2 &2 &9 &-1 \\ 1 &4 &0 &6\end{bmatrix}$$

2. *Real symmetric, positive definite matrices*. A matrix $A$ with real entires is symmetric if $A^T=A$, and *positive definite* if $\mathbf{x}^TA\mathbf{x} >0$ for all $\mathbf{x}\neq \mathbf{0}$. We remark here that $A$ being positive definite is sufficient to have a unique LU factorization.

3. *Complex  positive definite matrices*. A complex matrix that is positive definite is similar to a real positive definite matrix, except instead of $\mathbf{x}^TA\mathbf{x} > 0$ we have $\mathbf{x}^HA\mathbf{x} > 0$ for all $\mathbf{x}\neq \mathbf{0}$. Here, $\mathbf{x}^H$ means take the tranpose of $\mathbf{x}$ and set each entry to the complex conjugate of the corresponding entry in $\mathbf{x}$. $\mathbf{x}^H$ is also called the *conjugate transpose* of $\mathbf{x}$.



# Cholesky Factorization

If $A$ is a symmetric and positive definite (SPD) matrix with real entries, not only is it guaranteed to have an LU factorization, but it has a particularly nice factorization. To state this formally, if $A$ is SPD, then there exists an upper triangular matrix $R$ for which
$$ A = R^TR.$$
This factorization is called the **Cholesky factorization**, named after [André-Louis Cholesky](https://en.wikipedia.org/wiki/Andr%C3%A9-Louis_Cholesky), a French mathematician who developed this factorization while working as a surveryor, and fought in the first world war. If $A$ has complex entries, $R^T$ should be replaced with $R^H$.
To see this, let $A$ be SPD. We'll further assume $A$ has real entries. Since $A$ has an LU factorization, there exists a lower triangular matrix $\hat{L}$ such that $\hat{L}A$ is upper triangular. Since $A$ is symmetric, the same row operations that form $\hat{L}$, if applied to the *columns* of $A$, will result in the elimination of all entries above the diagonal. In other words,

$$\hat{L}A\hat{L}^T = D$$,

where $D$ is a diagonal matrix such that $d_{jj} = a_{jj}$. Since $A$ is positive definite, $a_{jj} > 0$ for each entry, and therefore $a_{jj} = \sqrt{a_{jj}}\sqrt{a_{jj}}$, If we write $\sqrt{D}$ has the diagonal matrix with diagonal entries $\sqrt{a_{jj}} = \sqrt{d_{jj}}$, and let $\hat{L}^{-1} = L$, then we have

$$ \hat{L}A\hat{L}^T = \sqrt{D}\sqrt{D} \iff A = L\sqrt{D}\sqrt{D}L^T= (L\sqrt{D})(L\sqrt{D})^T.$$

Thus, taking $R=(L\sqrt{D})^T$, we've shown that there exists an upper triangluar matrix $R$ for which $A = R^TR$.

It is possible to compute $R$ with about $n^3/3$ operations, which is about half that of the $LU$ factorization. The following method achieves this.

> Cholesky Factorization
Set $r_{11} = \sqrt{a_{11}}$. For $j=2,...,n$, set
$$
r_{ij} = a_{ij} - \sum_{k=1}^{i-1}r_{ki}r_{kj}, \hspace{1em} i=2,...,j-1
$$
$$
r_{jj} = \sqrt{a_{jj} - \sum_{k=1}^{j-1} r_{kj}^2}
$$

Note that if the upper index of the sum is less than the lower, we take the sum to be 0. This algorithm can be derived by assuming $A=R^TR$, and simply matching up terms beginning with $a_{11}$.

In [ ]:
#@title cholesky_example.m
%%writefile cholesky_example.m

% generate random 5x5 SPD matrix
n = 5; % set size of matrix
A = 5*rand(5); A = triu(A)'*triu(A); A

% compute Cholesky factorization using built in command
R_builtin = chol(A);

% compute Cholesky factorization manually
R = zeros(5,5);
R(1,1) = sqrt(A(1,1));

for j = 2:n
  for i = 1:j-1
    R(i,j) = (A(i,j) - R(1:i-1,i)'*R(1:i-1,j))/R(i,i);
  end
  R(j,j) = sqrt(A(j,j) - R(1:j-1,j)' * R(1:j-1,j));
end

% Compare results
R_builtin, R


In [ ]:
!octave -W cholesky_example.m

# Example 5.7 (Capillary networks)

On page 140 of our book, the authors demonstrate how we can model the flow of blood through a capillary bed with a linear system. We can then solve this system to determine the pressure at various points in the capillary bed. The idea is to model the capillary network as a collection of connected nodes, each node modeling the connection between three capillaries. By applying some fluid mechanics laws and conservation of mass at each node, we obtain the linear system $A\mathbf{p} = \mathbf{b}$, where
$\mathbf{p} = [p_1 \hspace{1em} p_2 \hspace{1em}\cdots \hspace{1em} p_{15}]^T$ is the vector of unknown pressures at each node, and the right-hand-side $\mathbf{b} = [-5/2 \hspace{1em} 0 \hspace{1em} \cdots \hspace{1em} 0]^T$ is known data.

The matrix $A$ of known coefficients is assembled in the cell below.

In [ ]:
#@title ex5_7.m
%%writefile ex5_7.m

% Step 1: Assemble A
d = [-0.25 -0.5 -0.5 -1*ones(1,4) -2*ones(1,8)];
u2 = [0.1 zeros(1,13)];
u3 = [0.1 0.2 zeros(1,11)];
u4 = [0 0.2 0.2 zeros(1,9)];
u5 = [0 0 0.2 0.4 zeros(1,7)];
u6 = [0 0 0 0.4 0.4 zeros(1,5)];
u7 = [0 0 0 0 0.4 0.4 zeros(1,3)];
u8 = [0 0 0 0 0 0.4 0.4 0];


A = diag(d) + diag(u2,1) + diag(u3,2) + diag(u4,3) + diag(u5,4) + diag(u6,5) + diag(u7,6) + diag(u8,7);
A(7,15) = 0.4;
A = tril(A',-1) + A ;

% Step 2: form right-hand-side b
b = [-5/2 zeros(1,14)]';

% Compute Cholesky. Note that -A is positive definite, but A is not, so we want to solve -Ax = -b.
R = chol(-A); b = -b;

% Solve R^Ty = b using forward subsitution
y = zeros(15,1); y(1) = b(1)/R(1,1);
for i = 2:15
  y(i) = (b(i) - y(1:i-1)' * R(1:i-1,i))/R(i,i);
end

% You can also use the built in command.
% This is the famous backlash operator in Matlab. It will apply the most efficient method to solve the given system.
%y = R' \ b;

% Solve Rx = y using backward subsitution
x = zeros(15,1); x(15) = y(15)/R(15,15);
for i = 14:-1:1
  x(i) = (y(i) - x(i+1:15)' * R(i,i+1:15)')/R(i,i);
end
% Or use the built in command
%x = R \ y
x

In [ ]:
!octave -W ex5_7.m

# Sparse Matrices

If you run spy(A) and spy(R) in Octave, you'll see a grid that shows all the nonzero entries in A and R respectively. You should note that the pattern of the nonzero entries in A and R are different, and that R has more nonzero entries.

Whenever there are a large number of entries in a matrix that are zero, we say that the matrix is **sparse**. The pattern of the nonzero entries is called the **sparsity pattern**. When this pattern is altered upon computing something like the LU or Cholesky decomposition, we say that this factorization generated *fill-in*. This is far from ideal, especially if we are solving a very large problem and memory is a concern. We'd like to preserve the structure and sparsity of our original matrix.

This can be accomplished by applying certain reordering algorithms to the matrix before computing a factorization. These are discussed a bit more on page 153ff of our book, and the authors provide some additional references for further reading. We'll also note here that when you have a sparse matrix, there is no need to store the zero entries. Storing only the nonzero entries frees up a significant amount of memory, and can be implemented with the MATLAB commands *sparse* or *spdiags*. Similar functions are in the SciPy linalg library.

# Pivoting

The last main topic of this lecture is how to resolve the issue we saw at the start. If we have a nonsingular matrix $A$, and we want to solve $A\mathbf{x} = \mathbf{b}$ efficiently, a reasonable idea is to compute the LU decomposition of $A$. However, even though $A$ is nonsingular, it may not have an LU decomposition. This is because the entry that we would like to use a pivot after the first step, the entry $a_{22}$, became a zero after the first round of Gaussian elimination. The solution is something you have likely seen if you've taken linear algebra: swap the rows so that the new $a_{22}$ entry is nonzero, and the proceed with Gaussian elimination.

Let's again consider
$$A = \begin{bmatrix} 1 &1 &4 \\ 2 &2 &1\\ 0 &1 &0\end{bmatrix}$$

One step of Gauss eliminiation lead us to

$$L_1A = \begin{bmatrix} 1 &0 &0 \\ -2 &1 &0\\ 0 &0 &1\end{bmatrix} \begin{bmatrix} 1 &1 &4 \\ 2 &2 &1\\ 0 &1 &0\end{bmatrix} = \begin{bmatrix} 1 &1 &4 \\ 0 &0 &-7\\ 0 &1 &0\end{bmatrix}.$$

If $a_{22}'$ is the entry in the second row and second column of $L_1A$, and $a_{32}'$ is the third row second column entry in $L_1A$, our algorithm from last time says to take
$$L_2 = \begin{bmatrix} 1 &0 &0 \\ 0 &1 &0 \\ 0 & -a_{32}'/a_{22}' &1\end{bmatrix}$$

and compute $L_2L_1A$. The problem is that $a_{22}' = 0$, and thus our algorithm breaks down. To get around this, we just swap the second and third rows. We do this by applying the elementary matrix corresponding to this row swap. These matrices are also known as *permutation matrices*. To obtain the elementary matrix that swaps row two and row three, start with the identity matrix, and swap these two rows. The result is the elementary matrix that swaps row two and row three:
$$ I = \begin{bmatrix} 1 &0 &0 \\ 0 &1 &0 \\ 0 &0 &1\end{bmatrix} \longrightarrow P_2 = \begin{bmatrix} 1 &0 &0 \\ 0 &0 &1 \\ 0 &1 &0\end{bmatrix}. $$

Applying $P_2$ to $L_1A$ gives

$$P_2L_1A = \begin{bmatrix} 1 &1 &4 \\ 0 &1 &0\\ 0 &0 &-7 \end{bmatrix}.$$

This particular example is nice because we don't even need another step of Gauss elimination since $P_2L_1A$ is upper triangular.

Let's stop and consider what we've actually done here. We haven't compute an LU factorization of $A$. Indeed, no such factorization exists since our algorithm without pivoting broke down. What we have actually shown is that

$$P_2A = LU,$$

where $L$ and a lower triangular matrix and $U$ is an upper triangular matrix. To see this, note that $P_2$ is invertible, and $P_2^{-1} = P_2$. So, writing

$$ P_2L_1A = U$$,

we can say that

$$P_2L_1P_2P_2A = (P_2L_1P_2)P_2A = U.$$ Recall that

$$L_1 =  \begin{bmatrix} 1 &0 &0 \\ -2 &1 &0\\ 0 &0 &1\end{bmatrix} $$

Since multiplying on the left by $P_2$ swaps rows 2 and 3, and multiplying on the right by $P_2$ swaps columns 2 and 3, we have

$$P_2L_1P_2 = \begin{bmatrix} 1 &0 &0 \\ -2 &1 &0\\ 0 &0 &1\end{bmatrix}  = L_1$$

Thus, letting $L = L_1^{-1}$, we've shown that $P_2A = LU$. The point is that what we have actually computed is $PA=LU$, where $P$ is a permutation matrix. A conseqeunce of this is that, when solving a linear system $A\mathbf{x}=\mathbf{b}$, our solve now takes the form

1. Multiply both sides of the linear system by $P$ to get $PA\mathbf{x}=P\mathbf{b}$.

2. Compute $PA = LU$.

3. Solve $L\mathbf{y} = P\mathbf{b}$.

4. Solve $U\mathbf{x} = \mathbf{y}$.

In practice, we don't necessarily know what $P$ is at the start of the solve. A more practical implemenation is given in the cell below. Note that pivoting is undertaken at every step. This is because, even if the diagonal entry is nonzero, we get a more numerically stable algorithm if we pivot so that the largest entry in the given column is in the pivot position. See the cell unstable_ex.m below to see what can happen if we apply LU without pivoting when we don't have any zero pivots, but we have a pivot that is much smaller than another element in the column.



In [ ]:
#@title lu_with_pivoting
%%writefile lu_with_pivoting.m

% we'll apply LU with pivoting to the particular matrix $A$
%A_og = [1 1 4; 2 2 1; 0 1 0];
A_og = 5*rand(5,5);
A = A_og;

[m,n] = size(A);

L = eye(n); P = eye(n);
for j = 1:n-1
  [~,pivot] = max(abs(A(j:n,j))); % find index of largest entry in column j below the diagonal.
  pivot=pivot+j-1;

  % swap rows so that largest entry is in position A(j,j)
  temp = A(j,:); A(j,:) = A(pivot,:); A(pivot,:) = temp;
  temp = P(j,:); P(j,:) = P(pivot,:); P(pivot,:) = temp;
  if j > 1
  pivot
  temp = L(pivot,1:j-1); L(pivot,1:j-1) = L(j,1:j-1); L(j,1:j-1) = temp;
  end


  % Now apply Gaussian elimination. We use loops here to emulate the book's algorithm.
  %for i = j+1:n
   % L(i,j) = A(i,j)/A(j,j) ;
    %for k = j+1:n
     % A(i,k) = A(i,k) - L(i,j)*A(j,k);
    %end
  %end

  % Alternatively, you can replace the loops with matrix operations.
  L(j+1:n,j) = A(j+1:n,j)/A(j,j);
  A(j+1:n,j+1:n) = A(j+1:n,j+1:n) - L(j+1:n,j)*A(j,j+1:n);

end

L,U=triu(A), P, P*A_og, L*U

disp("Using built in function")

[L,U,P] = lu(A_og)

L*U,P*A_og


In [ ]:
!octave -W lu_with_pivoting.m

## Pivoting for numerical stability

In the algorithm for LU with pivoting given in the cells above, we pivot whenever an element below $a_{jj}$ is larger in magnitude. We do this even if $a_{jj}\neq 0$. The result is a more numerically stable algorithm. In the cell below, we apply the LU algorithm without pivoting to the matrix

$$ A = \begin{bmatrix} 10^{-15} &1 \\ 1 &0.01 \end{bmatrix} $$

Since $A$ is 2x2, we only need one step of Gauss elimination. Taking

$$ L_1 = \begin{bmatrix} 1 &0 \\ -10^{15} &1 \end{bmatrix} $$, we obtain

$$L_1A = \begin{bmatrix} 10^{-15} &1 \\ 0 &0.01 - 10^{15} \end{bmatrix} $$

This is fine in exact arithmetic. In finite arithmetic, however, the distance between $-10^{15}$ and the next floating point number is $eps(-10^{15}) = 0.125$. So when we try and implement this in Octave, we'll find that

$$L_1A = \begin{bmatrix} 10^{-15} &1 \\ 0 &- 10^{15} \end{bmatrix} $$

Thus

$$ A = L_1^{-1}\begin{bmatrix} 10^{-15} &1 \\ 0 &- 10^{15} \end{bmatrix} = \begin{bmatrix} 1 &0 \\ 10^{15} &1 \end{bmatrix}\begin{bmatrix} 10^{-15} &1 \\ 0 &- 10^{15} \end{bmatrix} = \begin{bmatrix} 10^{-15} &1 \\ 1 &0\end{bmatrix} $$

This is definitely not the $A$ we started with. We can avoid this kind of issue by pivoting so that the largest entry in magintude in the column below the diagonal element is in the pivot position. You can experiment with the code in the cell below to see this example play out.



In [ ]:
#@title unstable_ex.m
%%writefile unstable_ex.m

format long

A = [1e-15 1 ; 1 0.01] % recall that eps(a) is the distance from a to the next floating point number greater than a

% the LU factorization only requires one step here.
L1 = [1 0 ; -A(2,1)/A(1,1) 1];

U = L1*A, L = inv(L1)

% difference should be the zero matrix
A - L*U

% See how built in lu functions handles A
disp("Using built in function with pivoting.")
[L,U,P] = lu(A);
L,U,P
P*A - L*U



In [ ]:
!octave -W unstable_ex.m

# Total pivoting

The pivoting technique discussed above, where we only swap rows, is known as *partial pivoting*. Though more expensive, there are situations where *total pivoting* is also useful. This is where we will swap rows and columns. It's worth noting that partial pivoting alone is quite effective in many cases, but total pivoting can help reduce fill-in of a sparse matrix.  

With total pivoting, we now have two permutation matrices $P$ and $Q$. The LU factorization obtained is
$$ PAQ = LU. $$

When solving the linear system $A\mathbf{x}=\mathbf{b}$ using total pivoting, we proceed through the following steps.

1. Multiply both sides on the left by $P$: $PA\mathbf{x}=P\mathbf{b}$.

2. The permutation matrix $Q$ is invertible, so we can write $PA\mathbf{x} = PAQ(Q^{-1}\mathbf{x}) = P\mathbf{b}$.

3. Since $PAQ = LU$, we now run through the following steps.  

  a. $L\mathbf{y} = P\mathbf{b}$,
  
  b. $U\mathbf{v} = \mathbf{y}$, and

  c. $\mathbf{x} = Q\mathbf{v}$.

